In [1]:
import os
import pandas as pd

audio_root_folder = './archive/data'
labels_csv = os.path.join(audio_root_folder, 'features_30_sec.csv')
df = pd.read_csv(labels_csv, header=0)
df.drop(df.loc[df.filename == 'jazz.00054.wav'].index, inplace=True)
df.head(5)

C:\Users\zheng\AppData\Local\Temp\ipykernel_18076\158973850.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [2]:
import numpy as np
import torch

RANDOM_SEED = RANDOM_STATE = 42

np.random.seed(RANDOM_SEED)
torch.random.manual_seed(RANDOM_SEED);

In [3]:
from sklearn.model_selection import train_test_split

filenames = df['filename']
labels = df['label']

files_train, files_val_test, labels_train, labels_val_test = train_test_split(
    filenames, labels, test_size=0.1, random_state=RANDOM_STATE
)

files_val, files_test, labels_val, labels_test = train_test_split(
    files_val_test, labels_val_test, test_size=0.5, random_state=RANDOM_STATE
)

In [4]:
from audio_toolbox.dataset import AudioOTFDataset

num_frames = 1290
label_encoding = 'Label'
scaling_strategy = None

datasets = {
    'train':
        AudioOTFDataset(
            root_folder=audio_root_folder,
            filenames=files_train.tolist(),
            labels=labels_train.tolist(),
            num_frames=num_frames,
            scaling_strategy=scaling_strategy,
            name='Training set',
            label_encoding=label_encoding,
            shuffle=True,
            random_state=RANDOM_STATE
        ),
    'val':
        AudioOTFDataset(
            root_folder=audio_root_folder,
            filenames=files_val.tolist(),
            labels=labels_val.tolist(),
            num_frames=num_frames,
            scaling_strategy=scaling_strategy,
            name='Validation set',
            label_encoding=label_encoding,
            shuffle=True,
            random_state=RANDOM_STATE
        ),
    'test':
        AudioOTFDataset(
            root_folder=audio_root_folder,
            filenames=files_test.tolist(),
            labels=labels_test.tolist(),
            num_frames=num_frames,
            scaling_strategy=scaling_strategy,
            name='Testing set',
            label_encoding=label_encoding,
            shuffle=True,
            random_state=RANDOM_STATE
        )
}

Processing for Testing set: 100%|██████████| 50/50 [00:06<00:00,  7.68it/s]


In [5]:
n_train, n_val, n_test = len(datasets['train']), len(datasets['val']), len(datasets['test'])
n_train, n_val, n_test

(899, 50, 50)

In [6]:
print(repr(datasets['train']))

====== Training set ======
Root folder: ./archive/data
Number of samples: 899
Shape of one sample: torch.Size([92880])
Number of classes: 10
Features:
	n_mfcc: 12
	n_chroma: 12
	n_derivatives: 2
Scaling strategy: None


In [7]:
import os

# Save the processed tensors to save time next time
os.makedirs('./processed_data/dl_data', exist_ok=True)
torch.save(datasets['train'].X, 'processed_data/dl_data/dl_modeling_train_data.pt')
print('Train data saved')
torch.save(datasets['train'].labels, 'processed_data/dl_data/dl_modeling_train_label.pt')
print('Train label saved')
torch.save(datasets['val'].X, 'processed_data/dl_data/dl_modeling_val_data.pt')
print('Val data saved')
torch.save(datasets['val'].labels, 'processed_data/dl_data/dl_modeling_val_label.pt')
print('Val label saved')
torch.save(datasets['test'].X, 'processed_data/dl_data/dl_modeling_test_data.pt')
print('Test data saved')
torch.save(datasets['test'].labels, 'processed_data/dl_data/dl_modeling_test_label.pt')
print('Test label saved')

Train data saved
Train label saved
Val data saved
Val label saved
Test data saved
Test label saved


In [8]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
pca = PCA(n_components=0.95)

train_scaled = scaler.fit_transform(datasets['train'].X)
val_scaled = scaler.transform(datasets['val'].X)
test_scaled = scaler.transform(datasets['test'].X)

train_data = torch.tensor(pca.fit_transform(train_scaled), dtype=torch.float)
val_data = torch.tensor(pca.transform(val_scaled), dtype=torch.float)
test_data = torch.tensor(pca.transform(test_scaled), dtype=torch.float)

In [9]:
train_data.shape, val_data.shape, test_data.shape

(torch.Size([899, 753]), torch.Size([50, 753]), torch.Size([50, 753]))

In [10]:
import os

# Save the processed tensors to save time next time
os.makedirs('./processed_data/dl_data_reduced', exist_ok=True)
torch.save(train_data, 'processed_data/dl_data_reduced/dl_modeling_train_data.pt')
print('Train data saved')
torch.save(datasets['train'].labels, 'processed_data/dl_data_reduced/dl_modeling_train_label.pt')
print('Train label saved')
torch.save(val_data, 'processed_data/dl_data_reduced/dl_modeling_val_data.pt')
print('Val data saved')
torch.save(datasets['val'].labels, 'processed_data/dl_data_reduced/dl_modeling_val_label.pt')
print('Val label saved')
torch.save(test_data, 'processed_data/dl_data_reduced/dl_modeling_test_data.pt')
print('Test data saved')
torch.save(datasets['test'].labels, 'processed_data/dl_data_reduced/dl_modeling_test_label.pt')
print('Test label saved')

Train data saved
Train label saved
Val data saved
Val label saved
Test data saved
Test label saved


In [11]:
import torch

train_data = torch.load('processed_data/dl_data_reduced/dl_modeling_train_data.pt')
val_data = torch.load('processed_data/dl_data_reduced/dl_modeling_val_data.pt')
test_data = torch.load('processed_data/dl_data_reduced/dl_modeling_test_data.pt')

train_label = torch.load('processed_data/dl_data_reduced/dl_modeling_train_label.pt')
val_label = torch.load('processed_data/dl_data_reduced/dl_modeling_val_label.pt')
test_label = torch.load('processed_data/dl_data_reduced/dl_modeling_test_label.pt')

train_data.shape, train_label.shape, val_data.shape, val_label.shape, test_data.shape, test_label.shape

(torch.Size([899, 753]),
 torch.Size([899]),
 torch.Size([50, 753]),
 torch.Size([50]),
 torch.Size([50, 753]),
 torch.Size([50]))

In [12]:
from torch.utils.data import TensorDataset

datasets = {
    'train': TensorDataset(train_data, train_label),
    'val': TensorDataset(val_data, val_label),
    'test': TensorDataset(test_data, test_label)
}

train_data.shape

torch.Size([899, 753])

In [13]:
import torch
from torch import nn

class SimpleLinearModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=64):
        super(SimpleLinearModel, self).__init__()
        self.linear = nn.Sequential(nn.Linear(input_size, hidden_dim),
                                    nn.ReLU(),
                                    nn.Linear(hidden_dim, hidden_dim),
                                    nn.ReLU(),
                                    nn.Linear(hidden_dim, output_size))

    def forward(self, x):
        # Assuming x is of shape (batch_size, input_size)
        return self.linear(x.view(x.shape[0], -1))

# Example usage:
input_size = train_data.size(1)
output_size = 10
batch_size = 32

model = SimpleLinearModel(input_size, output_size)


In [14]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-4  # Adjust the learning rate as needed
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [15]:
from audio_toolbox.trainer import ModelTrainer

trainer = ModelTrainer(datasets, model, loss_fn, optimizer, scheduler)

In [16]:
trainer_config = {
    'save': True,
    'num_epochs': 50
}

In [17]:
trainer.train(**trainer_config)

INFO:root:Epoch 0001, Learning rate: 0.000100, Training loss: 2.29735, Val loss: 2.50260, Test loss: 2.45054, Epoch time: 0.48153
INFO:root:Epoch 0002, Learning rate: 0.000100, Training loss: 2.26730, Val loss: 2.26801, Test loss: 2.25943, Epoch time: 0.30904
INFO:root:Epoch 0003, Learning rate: 0.000100, Training loss: 2.22474, Val loss: 2.07921, Test loss: 2.10522, Epoch time: 0.30056
INFO:root:Epoch 0004, Learning rate: 0.000100, Training loss: 2.18684, Val loss: 1.98731, Test loss: 1.93719, Epoch time: 0.28103
INFO:root:Epoch 0005, Learning rate: 0.000100, Training loss: 2.14136, Val loss: 1.82496, Test loss: 1.86597, Epoch time: 0.39868
INFO:root:Epoch 0006, Learning rate: 0.000050, Training loss: 2.10889, Val loss: 1.77824, Test loss: 1.77623, Epoch time: 0.32953
INFO:root:Epoch 0007, Learning rate: 0.000050, Training loss: 2.09041, Val loss: 1.70111, Test loss: 1.73170, Epoch time: 0.21284
INFO:root:Epoch 0008, Learning rate: 0.000050, Training loss: 2.06382, Val loss: 1.65057, 

In [18]:
train_res = trainer.predict(train_data)
test_res = trainer.predict(test_data)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(train_res, train_label)

0.67853170189099

In [20]:
accuracy_score(test_res, test_label)

0.26